## Predicting House Prices
### Regression Problem

In [1]:
from keras.datasets import boston_housing

/Users/ujjawalpathak/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
# prepare data

# data normalization
import numpy as np

mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [6]:
# build the model
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    return model

In [16]:
# k-fold validation
k = 0
num_val_sample = len(train_data) // 4
num_epochs = 100
all_scores = []

for i in range(k):
    print('preprocessing fold #', i)
    val_data = train_data[ i * num_val_sample: (i + 1) * num_val_sample]
    val_targets = train_targets[ i * num_val_sample: (i + 1) * num_val_sample]
    
    partial_train_data = np.concatenate([train_data[: i * num_val_sample],
                                        train_data[ (i + 1) * num_val_sample:]], axis=0)
    
    partial_train_targets = np.concatenate([train_targets[: i * num_val_sample],
                                           train_targets[ (i + 1) * num_val_sample:]],axis=0)
    
    model = build_model()
    model.fit(partial_train_data,partial_train_targets,
          epochs=num_epochs,
          batch_size=1,
          verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append[val_mae]